### Training

In [3]:
# import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
rst = eval( "['서울', '지방', '병무청']")
print(rst , type(rst))

['서울', '지방', '병무청'] <class 'list'>


#### CountVectorizer 예시

In [ ]:
sr = pd.Series(['hello hi','korea','hi kim'])
print(sr)
max_len = 100
vectorizer = CountVectorizer(max_features=10000)
sequences = vectorizer.fit_transform(sr).toarray()

0    hello hi
1       korea
2      hi kim
dtype: object


In [6]:
sequences

array([[1, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 1, 0]], dtype=int64)

In [7]:
vectorizer.vocabulary_

{'hello': 0, 'hi': 1, 'korea': 3, 'kim': 2}

==========================

In [8]:
# Load and preprocess the data
file_path = 'data/df_drop.csv'
df = pd.read_csv(file_path)

In [9]:
# Extract text and labels
texts = df['token_final'].apply(lambda x: ' '.join(eval(x)))  # Convert list-like strings to strings
labels = df['label']

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [ ]:
# Tokenize and pad sequences
max_len = 1000
vectorizer = CountVectorizer(max_features=10000)
sequences = vectorizer.fit_transform(texts).toarray()

# Convert sequences to tensor and truncate to max_len if necessary
sequences_tensor = torch.tensor(sequences[:, :max_len], dtype=torch.long)

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(sequences_tensor, labels_encoded, test_size=0.2, random_state=42)

In [ ]:
# Create PyTorch Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
# Define CNN model
class CNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_len):
        super(CNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim, 128, kernel_size=5)
        self.pool = nn.MaxPool1d(kernel_size=max_len - 4)
        self.fc1 = nn.Linear(128, 10)
        self.fc2 = nn.Linear(10, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x.long()).permute(0, 2, 1)
        x = self.conv1(x)
        x = self.pool(x).squeeze()
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

In [14]:
# Initialize model, loss function, and optimizer
vocab_size = 10000
embedding_dim = 128
model = CNNModel(vocab_size, embedding_dim, max_len)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
# Training loop
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for texts, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(texts)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

In [16]:
# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for texts, labels in test_loader:
            outputs = model(texts).squeeze()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {correct / total:.2f}")

In [17]:
# Train and evaluate the model
train_model(model, train_loader, criterion, optimizer, epochs=2)
evaluate_model(model, test_loader)

Epoch 1/2, Loss: 0.6396
Epoch 2/2, Loss: 0.6243
Test Accuracy: 0.70


In [ ]:
# Prediction for a single sentence
def predict_sentence(model, sentence):
    model.eval()
    tokens = vectorizer.transform([sentence]).toarray()
    input_tensor = torch.tensor(tokens[:, :max_len], dtype=torch.long)
    with torch.no_grad():
        output = model(input_tensor).squeeze()
        prediction = 'YES' if output > 0.5 else 'NO'
    return prediction

# Example prediction
example_sentence = """
억울한 일로 국민청원을 신청합니다.
 저는 **구치소 **교도관입니다.
 2015.10. 29. 이주 전 **구치소에는 독거실이 3개뿐이었는데 2014년 성북구 종암동 구역의 편입및 정신보건법의 개정으로 정신질환수용자가 갑자기 폭증을 하였습니다.
 그날도 존속살인 미수의 정신질환 수용자가 입소하였으나, 정신질환 수용자는 독거실에 수용하여야 하는데, 독거실이 없어 신입혼거실에 수용중 밤에 옆사람 목을 조르려 하였으나,팀원인 직원이 진압하지 못하여 제가 휴게시간에  휴식하지 못하고  진정시키다가 코피를 쏟고 호홉곤란이 발생하여 서울대 병원에서 여러 종합검사끝에 확장형심근병진단을 받았고 치료를 위하여 1주일 응급입원 검사끝에 갑상선항진증 진단을 받았습니다.  투약치료를 받던 중 2016년 6월 9일 심정지로 쓰러졌으나,행인의 도움으로 깨어나 인공제세동기 삽입시술을 받았으며, 인공제세동기는 8년마다 재시술을 받아야 하고, 상태가 심해지면 인공심장으로 교체수술을 받아야 한다고 합니다.연이은 철야로 당시 열악했던 근무상황에 대해 서울성모병원에서 업무상 과로로 인한  업무상 재해에 해당한다는 직업환경의학과의 소견서도 발급받았습니다.
저는 업무상 재해인정을 위한 공상소송중 행정소송 2심에서 공무상 요양신청서에 병명을 갑상선 항진이 아닌 과로와 연관이 없는 확장형심근병으로 기록하였다는 이유로 기각판결을 받았습니다.
그러나 확장형심근병으로 기록된 공무상 요양신청서는 행정심판 단계인 2017년 2월 제가 심정지후 회복을 위한 질병휴직에서 복귀한 뒤, 기관이 공무상요양신청서에 병명이 확장형심근병으로 인쇄된 용지를 주어 서명 날인만 한 후 제출된 것으로 당시 저는 심정지휴유증으로 인한 뇌손상으로 조용한 환경이 아니면 기억력에 장애를 격는 휴유증에 시달리고 있어서 제대로 항의도 못하였고, 또한 재해 발생경위서도 제가 변호사를 선임하여 작성한 문서를 소장이 결제하지 않고 반려하여[보안사항을 외부 변호사에게 작성시켰다는 이유로] 행정직원에게 다시 작성하게 하였는데,주요과로내용이 다 빠지고 기각되기 딱 좋도록 작성된 재해발생경위서를 만들어 주어 , 제가 난감해 하니 변호사님이 '소송에서 다투면 되니 일단 접수를 하라'고 하여 접수하였으나, 연이은 행정심판 단계에서 다 패소하였으며, 행정소송 2심에서는 정작 기관에서 준 병명이 인쇄된 요양신청서로 기각판결을 받고 말았습니다  그 신청서는 당시 총무과 직원이 저보고 집에서 작성 제출하라고 메일로 보내줬으나, 집 프린터가 검정색잉크는 다쓰고 빨간색으로 출력되어 작성하지 못하고 이튿날 총무과에서 준 용지에 날짜와 서명만 쓰고 제출하였습니다.
 저는  팀장인 제책임을 다하여 조직과 동료에 민페를 안 끼치려 팀원의 근무를 휴계시간에 쉬지 못하고 연이어 대신하다가 과로로 심장이 망가져 쓰러졌는데, 기관은 이 소송이 기관장의 근평을 감점시켜 승진에 지장을 준다는 이유로 갖은 방법으로 자료제출등 소송을 방해하였으며, 소송을 무효화 시키기 위하여 30년 근무동안 제 과실로 인한 근무보고서는  한장을 써본 적이 없는데 심지어 징계건도 아닌 사유로 징계까지 주고 조사과정에서 강압으로 3년전 제가 쓰러진 상황을 위조하여 소송을 무효시키려는 시도까지 하였습니다.  공무상 요양소송을 패소시키기 위해, 기관에서 중앙통제실에 소송 관계자를 배치하여 저를 불법 사찰하여[소송 하였다는 이유로] 근무중 문서를 출력하기 위해 3분정도 프린터실에 간 것까지 근무지 이탈로 근무보고서를 제출요구 하여 징계에 넘겼으며[견책]4. 2019. 9. 3. 아침 7시 40분 점검 시간에 저에게 팀장 이oo가 서류제출을 요구해 [통상 점검시간에는 서류제출을 요구하지 않음] 제출하고, 미처 점검준비를 하지 못하고 사소출역방인 9. 10방 개문이 지적되어, 근무보고서를 제출하라하여 제출하고, 잠시후 행정계장에게 전화가 와서 교육 불참을 이유로 다시 근무보고서를 제출하라 하여 제출하니, 팀장이 팀사무실로 불러서 내려가니 개문, 교육불참을 이유로 징계통고를 하여 '부당하다, 소청하겠다'고 답변후 채증하여 녹음하니, 잠시후 조사실로 가라고 교대자를 보내서, 근무자실 화장실에서 제 변호사에게 ' 부당하게 조사 감금하려 한다, 조사를 거부해도 되냐', 고 문의통화[통화내역 첨부]를 하였더니, 저를 조사실로 끌고 가려고 팀장 이**가 왔다가 통화목소리를 듣고, '핸드폰 통화도 징계하겠다'고 하여 견책 징계를 받게 되었으며[연이은 근무보고서제출 및 호출로 보관함에 맡길 수 없었음],심정지후 제세동기 시술로 몸 상태도 안좋은 상태[가슴통증]에서 공로연수갈 계획이었으나, 징계로 공로연수및 명예퇴직이 불가능하게 만들었습니다.  2019. 9. 3. 21시까지 조사를 끝낸 후, 2019. 9. 19. 퇴근길에 모르는 부재중 전화가 있어서 통화해 보니 조사계장과 행정계장으로 저에게 3년전 심정지때의 정황을 물어서, 그걸 왜 묻냐고 하니 도와주려 한다고 하여, '퇴근길에~라고, 답변하니, '무슨 퇴근길이냐?~,'며 소리를 버럭지르며 심정지시의 정황을 위조하려 시도하여, 화가나 같이 소리를 지르고 주장하다가, 다음날 조사실로 오라고 하여 통화를 끊고, 변호사에게 상황을 전하니 '앞으로 모든 조사를 거부하고 강제 연행하려 하면 112에 신고하여 보호받으라'고 하여 초긴장 상태로 근무하다가 변론이 종결되었습니다. 직원들이 저의 징계소식을 듣고 찾아와 '무슨일로 징계냐'고 물어서, 교육 불참및 점검시 개문이라 하였더니, '어, 나도 교육 안갔는데~, 서로 답변하였으며, 헌법에도 규정된 보편적 타당성이 결여된 개별적 처분적법률의 행사로 직권 남용입니다.
행정소송의 위법한 패소로 치료의 길이 난감할 뿐 입니다.
공정한 제대로된 소송이 보장된 법치의 길이 열리길 기도할 뿐입니다.
"""
print(f"Prediction for '{example_sentence}': {predict_sentence(model, example_sentence)}")

Prediction for '
억울한 일로 국민청원을 신청합니다.
 저는 **구치소 **교도관입니다.
 2015.10. 29. 이주 전 **구치소에는 독거실이 3개뿐이었는데 2014년 성북구 종암동 구역의 편입및 정신보건법의 개정으로 정신질환수용자가 갑자기 폭증을 하였습니다.
 그날도 존속살인 미수의 정신질환 수용자가 입소하였으나, 정신질환 수용자는 독거실에 수용하여야 하는데, 독거실이 없어 신입혼거실에 수용중 밤에 옆사람 목을 조르려 하였으나,팀원인 직원이 진압하지 못하여 제가 휴게시간에  휴식하지 못하고  진정시키다가 코피를 쏟고 호홉곤란이 발생하여 서울대 병원에서 여러 종합검사끝에 확장형심근병진단을 받았고 치료를 위하여 1주일 응급입원 검사끝에 갑상선항진증 진단을 받았습니다.  투약치료를 받던 중 2016년 6월 9일 심정지로 쓰러졌으나,행인의 도움으로 깨어나 인공제세동기 삽입시술을 받았으며, 인공제세동기는 8년마다 재시술을 받아야 하고, 상태가 심해지면 인공심장으로 교체수술을 받아야 한다고 합니다.연이은 철야로 당시 열악했던 근무상황에 대해 서울성모병원에서 업무상 과로로 인한  업무상 재해에 해당한다는 직업환경의학과의 소견서도 발급받았습니다.
저는 업무상 재해인정을 위한 공상소송중 행정소송 2심에서 공무상 요양신청서에 병명을 갑상선 항진이 아닌 과로와 연관이 없는 확장형심근병으로 기록하였다는 이유로 기각판결을 받았습니다.
그러나 확장형심근병으로 기록된 공무상 요양신청서는 행정심판 단계인 2017년 2월 제가 심정지후 회복을 위한 질병휴직에서 복귀한 뒤, 기관이 공무상요양신청서에 병명이 확장형심근병으로 인쇄된 용지를 주어 서명 날인만 한 후 제출된 것으로 당시 저는 심정지휴유증으로 인한 뇌손상으로 조용한 환경이 아니면 기억력에 장애를 격는 휴유증에 시달리고 있어서 제대로 항의도 못하였고, 또한 재해 발생경위서도 제가 변호사를 선임하여 작성한 문서를 소장이 결제하지 않고 반려하여[보안사항을 외부 변호사에게 작성시켰다는 이유로] 행정직원에게 다시 작성하게 하